In [1]:
import math
import pandas as pd
import numpy as np

In [11]:
def compute_precision_at_k(l, k):
    l = l[:k]
    return sum(l)/k

def compute_map_at_k(l, k):
    ap = []
    for s in range(1, k+1):
        ap.append(compute_precision_at_k(l, s))
    return sum(ap)/len(ap)
def compute_ndcg(l, k):
    l = l[:k]
    dcg = sum(([(math.pow(2, l[i]) - 1) / math.log(i + 2, 2) for i in range(len(l))]))
    sorted_l = sorted(l, reverse = True)
    idcg = sum(([(math.pow(2, sorted_l[i]) - 1) / math.log(i + 2, 2) for i in range(len(sorted_l))]))
    if (idcg > 0):
        return dcg/idcg
    else:
        return float(0)
def compute_mrr(l, k):
    return sum([l[i]*float(1)/float(i + 1) for i in range(len(l))])/k

In [3]:
df = pd.read_csv("d:/downloads/json/ehealthforum/trac/validation_df.txt", sep = "\t")

In [4]:
df.columns

Index(['Unnamed: 0', 'query_category', 'query_thread', 'query_text',
       'query_annotations', 'typ_dsyn', 'typ_patf', 'typ_sosy', 'typ_dora',
       'typ_fndg', 'typ_menp', 'typ_chem', 'typ_orch', 'typ_horm', 'typ_phsu',
       'typ_medd', 'typ_bhvr', 'typ_diap', 'typ_bacs', 'typ_enzy', 'typ_inpo',
       'typ_elii', 'document_category', 'document_thread', 'document_text',
       'document_is_doctor_reply', 'document_number_votes_h',
       'document_number_votes_s', 'document_number_votes_t',
       'document_user_status', 'document_annotations', 'd_typ_dsyn',
       'd_typ_patf', 'd_typ_sosy', 'd_typ_dora', 'd_typ_fndg', 'd_typ_menp',
       'd_typ_chem', 'd_typ_orch', 'd_typ_horm', 'd_typ_phsu', 'd_typ_medd',
       'd_typ_bhvr', 'd_typ_diap', 'd_typ_bacs', 'd_typ_enzy', 'd_typ_inpo',
       'd_typ_elii', 'relationships_list', 'bm25_relevant', 'bm25_score',
       'predicted_prob'],
      dtype='object')

In [6]:
col = ["query_thread", 'document_text', 'bm25_score', 'predicted_prob']

In [7]:
new_df = df[col]
new_df["thresh_0.0"] = new_df['predicted_prob']>0.0
new_df["thresh_0.25"] = new_df['predicted_prob']>0.25
new_df["thresh_0.5"] = new_df['predicted_prob']>0.5
new_df["thresh_0.75"] = new_df['predicted_prob']>0.75
new_df["thresh_0.9"] = new_df['predicted_prob']>0.9
new_df["thresh_1.0"] = new_df['predicted_prob']>1.0

d:\documents\python\healthdata\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\documents\python\healthdata\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
d:\documents\python\healthdata\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [8]:
prec_df = pd.DataFrame(columns=['query_thread','thresh_0.0', 'thresh_0.25', 'thresh_0.5', 'thresh_0.75', 'thresh_0.9', 'thresh_1.0'])

In [14]:
cnt = 0
labels = ['thresh_0.0', 'thresh_0.25', 'thresh_0.5', 'thresh_0.75', 'thresh_0.9', 'thresh_1.0']
for query, group in new_df.groupby(["query_thread"]):
    sorted_df = group.sort_values(by=['bm25_score'], ascending=False)
    res = []
    for lb in labels:
        top_k_res = sorted_df[lb].head(n = 10).tolist()
        res.append(compute_precision_at_k(top_k_res, 10))
    prec_df.loc[cnt] = [query]+res
    cnt += 1

In [54]:
prec_df.mean()

query_thread    22127.7520
thresh_0.0          1.0000
thresh_0.25         0.7794
thresh_0.5          0.7794
thresh_0.75         0.7794
thresh_0.9          0.2532
thresh_1.0          0.0000
dtype: float64